In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup

query = input("Enter keywords: ")
q_str = '-'.join(query.split(' '))
base_url = "https://www.bayt.com"

# Use a headless browser (requires a compatible web driver, like chromedriver)
driver = webdriver.Chrome()
page_no = 1

# Open a CSV file for writing
csv_file_path = 'job_data.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)
    # Write header row
    csv_writer.writerow(['Title', 'Description', 'Location','Link'])

    while True:
        url = f"https://www.bayt.com/en/pakistan/jobs/{q_str}-jobs/?page={page_no}"
        driver.get(url)
        
       
        wait = WebDriverWait(driver, timeout=10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'has-pointer-d')))

        # Parse the page content
        soup = BeautifulSoup(driver.page_source, "html.parser")
        div_elements = soup.find_all('li', class_='has-pointer-d')

        # Process the data as needed
        for div_element in div_elements:
            h2_tag = div_element.find("h2")
            anchor_tag = div_element.find("a")
            div_tag = div_element.find("div", class_="jb-descr m10t t-small")
            dis_tag = div_element.find("span")

            if h2_tag and anchor_tag and div_tag and dis_tag:
                href = anchor_tag.get("href")
                h2_text = h2_tag.text.strip()
                div_text = div_tag.text.strip()
                dis_text = dis_tag.text.strip()
                full_url = base_url + href

                # Write job details to the CSV file
                csv_writer.writerow([h2_text, div_text, dis_text, full_url,])

        print("Page NO " + str(page_no))

        # Check if the "Load More" button is present
        load_more_button = None
        try:
            load_more_button = driver.find_element(By.XPATH, '//*[@id="pagination"]/li[6]')
        except NoSuchElementException:
            pass

        # Check if the "Next" button is present
        next_button = None
        try:
            next_button = driver.find_element(By.XPATH, '//*[@id="pagination"]/li[4]')
        except NoSuchElementException:
            pass
        
        if load_more_button:
            if load_more_button and load_more_button.is_displayed():
                driver.execute_script("arguments[0].scrollIntoView(true);", load_more_button)
                
                driver.execute_script("arguments[0].click();", load_more_button)
            else:
                print("No more results. Exiting the loop.")
                break    
        else:
            if next_button and next_button.is_displayed():
                
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
               
                driver.execute_script("arguments[0].click();", next_button)
            else:
                print("No more results. Exiting the loop.")
                break 

        page_no += 1 


print(f"Job data has been saved to {csv_file_path}")


driver.quit()


In [ ]:
import pandas as pd
df=pd.read_csv(r'C:\Users\Asad Rajar\Desktop\Selenium\job_data.csv')
df
